In [37]:
import json

from tvDatafeed import TvDatafeed

tv = TvDatafeed()

In [39]:
def write_raw_data(raw_data, symbol):
    with open(f"{symbol}.json", 'w') as file:
        file.write(raw_data)

def raw_data_to_json(raw_data):
    results = raw_data.split("\n")
    sections = []
    for result in results:
        parsed = tv.parse_m_format(result)
        sections.append(parsed)
    
    return sections

def flatten_list(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_list(item))
        elif isinstance(item, dict):
            if "v" in item:
                flattened.append(item["v"])
            if "p" in item:
                if "v" in item["p"][1]:
                    flattened.append(item["p"][1]["v"])
        else:
            flattened.append(item)
    return flattened

def get_financial_data(
    symbol: str,
    exchange: str,
) -> dict:
    """get financial ratios

    Args:
        symbol (str): symbol name
        exchange (str, optional): exchange, not required if symbol is in format EXCHANGE:SYMBOL. Defaults to None.

    Returns:
        dict
    """
    symbol = tv.format_symbol(symbol=symbol, exchange=exchange)

    tv.create_connection()

    tv.send_set_auth_token()
    tv.send_quote_create_session_msg()
    tv.send_message(
        "quote_add_symbols",
        [tv.session, symbol],
    )
    raw_data = tv.receive_data(sentinel="quote_completed")

    return raw_data

def format_float(value, decimal_places=2, default="N/A"):
    """
    Formats a float value to the specified number of decimal places.
    Returns the default string if the value is None.
    
    :param value: The float value to format
    :param decimal_places: The number of decimal places to display
    :param default: The default string to return if value is None
    :return: A formatted string or the default string
    """
    if value is None:
        return default
    format_spec = f".{decimal_places}f"
    return f"{value:{format_spec}}"

financial_data_raw = get_financial_data("BATS", "LSE")

sections = raw_data_to_json(financial_data_raw)
write_raw_data(json.dumps(sections, indent=4), "sections")

flattened_sections = flatten_list(sections)

write_raw_data(json.dumps(flattened_sections, indent=4), "flattened_sections")

flattened_dict = {key: value for d in flattened_sections for key, value in d.items()}
write_raw_data(json.dumps(flattened_dict, indent=4), "flattened_dict")

In [42]:
def print_financials(financials_dict):

    share_price = financials_dict.get('lp', None)
    currency = financials_dict.get('currency_code', None)
    shares_outstanding = financials_dict.get('total_shares_outstanding_current', None)
    average_volume = financials_dict.get('average_volume', None)
    market_cap = financials_dict.get('market_cap_calc', None)
    country_code = financials_dict.get('country_code', None)
    isin = financials_dict.get('isin', None)
    exhange = financials_dict.get('exchange', None)
    figi = financials_dict.get('figi', None)
    figi_exchange = figi.get("exchange-level", None) if figi else None
    figi_composite = figi.get("country-composite", None) if figi else None
    beta_1yr = financials_dict.get('beta_1_year', None)
    description = financials_dict.get('description', None)

    print(f"Share Price:          {share_price} ({currency})")
    print(f"Shares Outstanding:   {shares_outstanding}")
    print(f"Average Volume:       {average_volume}")
    print(f"Market Cap:           {market_cap}")
    print(f"Country Code:         {country_code}")
    print(f"ISIN:                 {isin}")
    print(f"Exchange:             {exhange}")
    if figi:
        print(f"FIGI Exchnage:        {figi_exchange}")
        print(f"FIGI Composite:       {figi_composite}")
    print(f"Beta 1 Year:          {format_float(beta_1yr)}")
    print(f"Description:          {description}")

    # Valuation ratios
    price_earnings_ratio = financials_dict.get("price_earnings", None)
    price_sales_ratio = financials_dict.get("price_sales_current", None)
    price_cash_flow_ratio = financials_dict.get("price_cash_flow_current", None)
    price_book_ratio = financials_dict.get("price_book_current", None)
    ev = financials_dict.get("enterprise_value_current", None)
    ev_ebitda = financials_dict.get("enterprise_value_ebitda_current", None)
    currency = financials_dict.get("currency", None)

    print("Valuation Ratios")
    print(f"  P/E:        {format_float(price_earnings_ratio)}")
    print(f"  P/S:        {format_float(price_sales_ratio)}")
    print(f"  P/B:        {format_float(price_book_ratio)}")
    print(f"  P/CF:       {format_float(price_cash_flow_ratio)}")
    print(f"  EV:         {format_float(ev)} ({currency})")
    print(f"  EV/EBITDA:  {format_float(ev_ebitda)}")

    # Profitability ratios
    return_on_assets = financials_dict.get("return_on_assets_current", None)
    return_on_equity = financials_dict.get("return_on_equity_current", None)
    return_on_invested_capital = financials_dict.get("return_on_invested_capital_current", None)
    gross_margin = financials_dict.get("gross_margin_current", None)
    operating_margin = financials_dict.get("operating_margin_current", None)
    ebitda_margin = financials_dict.get("ebitda_margin_current", None)
    net_margin = financials_dict.get("net_margin_current", None)

    print("\nProfitability Ratios")
    print(f"  Return on Assets:               {format_float(return_on_assets)}%")
    print(f"  Return on Equity:               {format_float(return_on_equity)}%")
    print(f"  Return on Invested Capital:     {format_float(return_on_invested_capital)}%")
    print(f"  Gross Margin:                   {format_float(gross_margin)}%")
    print(f"  Operating Margin:               {format_float(operating_margin)}%")
    print(f"  EBITDA Margin:                  {format_float(ebitda_margin)}%")
    print(f"  Net Margin:                     {format_float(net_margin)}%")


    # Liquidity ratios
    quick_ratio = financials_dict.get("quick_ratio", None)
    current_ratio = financials_dict.get("current_ratio", None)
    inventory_turnover = financials_dict.get("invent_turnover_current", None)
    asset_turnover = financials_dict.get("asset_turnover_current", None)

    print("\nLiquidity Ratios")
    print(f"  Quick Ratio:         {format_float(quick_ratio)}")
    print(f"  Current Ratio:       {format_float(current_ratio)}")
    print(f"  Inventory Turnover:  {format_float(inventory_turnover)}")
    print(f"  Asset Turnover:      {format_float(asset_turnover)}")

    # Solvency ratios
    debt_assets_ratio = financials_dict.get("debt_to_asset_current", None)
    debt_equity_ratio = financials_dict.get("debt_to_equity_current", None)
    long_term_debt_assets_ratio = financials_dict.get("long_term_debt_to_assets_current", None)
    long_term_debt_equity_ratio = financials_dict.get("long_term_debt_to_equity_current", None)

    print("\nSolvency Ratios")
    print(f"  Debt to Assets Ratio:      {format_float(debt_assets_ratio)}")
    print(f"  Debt to Equity Ratio:      {format_float(debt_equity_ratio)}")
    print(f"  LT Debt to Assets Ratio:   {format_float(long_term_debt_assets_ratio)}")
    print(f"  LT Debt to Equity Ratio:   {format_float(long_term_debt_equity_ratio)}")

print_financials(flattened_dict)

Share Price:          3412.0 (GBX)
Shares Outstanding:   2200558021
Average Volume:       5265864.6
Market Cap:           7504867381468.9375
Country Code:         GB
ISIN:                 GB0002875804
Exchange:             LSE
FIGI Exchnage:        BBG000BG9N74
FIGI Composite:       BBG000BG9MW8
Beta 1 Year:          0.60
Description:          BRITISH AMERICAN TOBACCO ORD GBP0.25
Valuation Ratios
  P/E:        N/A
  P/S:        2.89
  P/B:        1.39
  P/CF:       8.46
  EV:         108472632727.00 (GBP)
  EV/EBITDA:  8.54

Profitability Ratios
  Return on Assets:               -10.46%
  Return on Equity:               -21.96%
  Return on Invested Capital:     -14.13%
  Gross Margin:                   58.19%
  Operating Margin:               34.63%
  EBITDA Margin:                  47.76%
  Net Margin:                     36.23%

Liquidity Ratios
  Quick Ratio:         0.58
  Current Ratio:       0.88
  Inventory Turnover:  1.68
  Asset Turnover:      0.20

Solvency Ratios
  Debt to A

### Attempting to get multiple symbols in one connection. Needs work

In [47]:
def receive_data2(self, sentinel):
    print("receive_data2")
    raw_data = ""
    while True:
        try:
            result = self.ws.recv()
            raw_data = raw_data + result + "\n"
        except Exception as e:
            print(f"Error: {e}")
            break

        if sentinel in result:
            break

    return raw_data

TvDatafeed.receive_data = receive_data2

def get_multi_financial_data(symbols: list[str]) -> dict:
    """get financial ratios

    Args:
        symbol (str): symbol name
        exchange (str, optional): exchange, not required if symbol is in format EXCHANGE:SYMBOL. Defaults to None.

    Returns:
        dict
    """

    tv.create_connection()

    tv.send_set_auth_token()
    tv.send_quote_create_session_msg()

    for symbol in symbols:
        tv.send_message(
            "quote_add_symbols",
            [tv.session, symbol],
        )

    raw_data = tv.receive_data(sentinel="quote_completed")

    return raw_data


symbols = [
    tv.format_symbol(symbol="BATS", exchange="LSE"),
    tv.format_symbol(symbol="IMB", exchange="LSE"),
    tv.format_symbol(symbol="JEMA", exchange="LSE"),
    tv.format_symbol(symbol="NWG", exchange="LSE"),
]

financial_data_raw = get_multi_financial_data(symbols)
write_raw_data(financial_data_raw, "multi_financial_data")

# not working - need to rework for multiple symbols
multi_sections = raw_data_to_json(financial_data_raw)
write_raw_data(json.dumps(multi_sections, indent=4), "multi_sections")

In [48]:
def flatten_lists(nested_list):
    flattened = []
    for item in nested_list:
        if isinstance(item, list):
            flattened.extend(flatten_lists(item))
        else:
            flattened.append(item)
    return flattened

flattened_multi_sections = flatten_lists(multi_sections)
write_raw_data(json.dumps(flattened_multi_sections, indent=4), "flattened_multi_sections")

In [49]:
symbol_data = {}

for fms in flattened_multi_sections:
    if "p" in fms:
        if "v" in fms["p"][1]:
            symbol = fms["p"][1]["n"]
            data = fms["p"][1]["v"]
            symbol_data.setdefault(symbol, []).append(data)

In [50]:
display(symbol_data.keys())

for key in symbol_data.keys():
    display(key)
    display(symbol_data[key][0])

    # for data in symbol_data[key]:
    #     display(data)
# display(symbol_data["LSE:BATS"][2])

dict_keys(['LSE:BATS', 'LSE:NWG', 'LSE:JEMA', 'LSE:IMB'])

'LSE:BATS'

{'average_volume': 5265864.6,
 'first_bar_time_1m': 1233561600,
 'measure': 'price',
 'chp': 0.6,
 'visible-plots-set': 'ohlcv',
 'original_name': 'LSE_DLY:BATS',
 'earnings_release_next_date': 1739430000,
 'regular_close_time': 1739214991,
 'total_revenue': 27334000000,
 'source-logoid': 'source/LSE',
 'logoid': 'british-american-tobacco',
 'days_to_maturity': None,
 'lp': 3410.2178,
 'high_price': 3416,
 'language': 'en',
 'last_annual_eps': -6.4657,
 'volume': 821937.0,
 'pointvalue': 1,
 'rch': None,
 'all_time_high': 5643.6001,
 'provider_id': 'ice',
 'rtc_time': None,
 'minmov': 1,
 'source2': {'country': 'GB',
  'description': 'London Stock Exchange',
  'exchange-type': 'exchange',
  'id': 'LSE',
  'name': 'London Stock Exchange',
  'url': 'https://www.londonstockexchange.com'},
 'all_time_low_day': 956707200,
 'short_description': 'BRITISH AMERICAN TOBACCO ORD GBP0.25',
 'currency-logoid': 'country/GB',
 'variable_tick_size': '0.0001 0.1 0.0001 0.2 0.0001 0.5 0.0002 1 0.0005 2 

'LSE:NWG'

{'trade_loaded': True,
 'bid_size': 1938.0,
 'bid': 444.7,
 'ask_size': 4477.0,
 'ask': 444.8}

'LSE:JEMA'

{'first_bar_time_1m': 1669366320,
 'measure': 'price',
 'chp': -0.2,
 'visible-plots-set': 'ohlcv',
 'original_name': 'LSE_DLY:JEMA',
 'regular_close_time': 1739205308,
 'total_revenue': 3442000,
 'source-logoid': 'source/LSE',
 'logoid': 'jpmorgan',
 'days_to_maturity': None,
 'lp': 205.588,
 'high_price': 205.6273,
 'language': 'en',
 'price_earnings_ttm': 32.9073482428115,
 'last_annual_eps': 0.0626,
 'volume': 20456,
 'pointvalue': 1,
 'rch': None,
 'all_time_high': 894,
 'provider_id': 'ice',
 'rtc_time': None,
 'minmov': 1,
 'source2': {'country': 'GB',
  'description': 'London Stock Exchange',
  'exchange-type': 'exchange',
  'id': 'LSE',
  'name': 'London Stock Exchange',
  'url': 'https://www.londonstockexchange.com'},
 'all_time_low_day': 1655683200,
 'short_description': 'JPMORGAN EMERGING EMEA SEC PLC ORD GBP0.01',
 'currency-logoid': 'country/GB',
 'variable_tick_size': '0.0002 0.1 0.0005 0.2 0.001 0.5 0.002 1 0.005 2 0.01 5 0.02 10 0.05 20 0.1 50 0.2 100 0.5 200 1 500 2 1

'LSE:IMB'

{'average_volume': 2023506.2,
 'first_bar_time_1m': 1233561600,
 'measure': 'price',
 'chp': 0.82,
 'visible-plots-set': 'ohlcv',
 'original_name': 'LSE_DLY:IMB',
 'earnings_release_next_date': 1747742400,
 'regular_close_time': 1739214753,
 'total_revenue': 18486000000,
 'source-logoid': 'source/LSE',
 'logoid': 'imperial-brands',
 'days_to_maturity': None,
 'lp': 2880.55,
 'high_price': 2888.0,
 'language': 'en',
 'price_earnings_ttm': 9.458193979933112,
 'last_annual_eps': 2.99,
 'volume': 290572.0,
 'pointvalue': 1,
 'rch': None,
 'all_time_high': 4154,
 'provider_id': 'ice',
 'rtc_time': None,
 'minmov': 1,
 'source2': {'country': 'GB',
  'description': 'London Stock Exchange',
  'exchange-type': 'exchange',
  'id': 'LSE',
  'name': 'London Stock Exchange',
  'url': 'https://www.londonstockexchange.com'},
 'all_time_low_day': 890006400,
 'short_description': 'IMPERIAL BRANDS PLC GBP0.10',
 'currency-logoid': 'country/GB',
 'variable_tick_size': '0.0001 0.1 0.0001 0.2 0.0001 0.5 0.